In [1]:
import requests
import psycopg2
from datetime import datetime, timedelta
import time

In [2]:
start_date = datetime.strptime("2023-06-12", "%Y-%m-%d")
end_date = datetime.strptime("2023-06-12", "%Y-%m-%d")

date_list = []
current_date = start_date

while current_date <= end_date:
    formatted_date = current_date.strftime("%Y-%m-%d")
    date_list.append(formatted_date)
    current_date += timedelta(days=1)


In [3]:
job_list = ['arrange', 'in', 'out', 'retin', 'retout', 'trans', 'SHOP_REQ']
# arrange 조정
# in 입고
# out 출고
# retin 반품입고
# retout 반품출고
# trans 배송처리
# SHOP_REQ 매장출고

In [4]:
def get_stock_log(start_date, end_date, job, page):


    partner_key = '73c6ac2e3366905793ab35d5a222d572'
    domain_key = '73c6ac2e33669057718abd505d76aa87366d3eed6643997457a0f9a03797aa9f'
    # start_date = '2023-01-01'
    # end_date = '2023-05-21'
    # job = 'in'
    limit = '100'
    

    url = f'http://api2.cloud.ezadmin.co.kr/ezadmin/function.php?partner_key={partner_key}&domain_key={domain_key}&action=get_stock_tx_info&start_date={start_date}&end_date={end_date}&job={job}&page={page}&limit={limit}'
    res = requests.get(url)

    if res.status_code == 200:
        return res.json()['data']
    else:
        return ValueError


In [5]:
def save_stock_log(cursor, data_list):

    query = """
        INSERT INTO "stock_log" (seq, crdate, product_id, job, qty, stock, bad, owner, memo, supply_code, supply_price, name, options, barcode, origin, brand, enable_sale, org_price, org_price_qty, order_seq, sheet, sheet_name, product_memo) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) 
        ON CONFLICT (seq) DO UPDATE 
        SET crdate = EXCLUDED.crdate, product_id = EXCLUDED.product_id, job = EXCLUDED.job, qty = EXCLUDED.qty, stock = EXCLUDED.stock, 
            bad = EXCLUDED.bad, owner = EXCLUDED.owner, memo = EXCLUDED.memo, supply_code = EXCLUDED.supply_code, 
            supply_price = EXCLUDED.supply_price, name = EXCLUDED.name, options = EXCLUDED.options, barcode = EXCLUDED.barcode, 
            origin = EXCLUDED.origin, brand = EXCLUDED.brand, enable_sale = EXCLUDED.enable_sale, org_price = EXCLUDED.org_price, 
            org_price_qty = EXCLUDED.org_price_qty, order_seq = EXCLUDED.order_seq, sheet = EXCLUDED.sheet, 
            sheet_name = EXCLUDED.sheet_name, product_memo = EXCLUDED.product_memo;
    """

    for d in data_list:
        cursor.execute(query, d)


In [6]:
conn = psycopg2.connect(host='115.68.228.168', user='postgres', password='cldngks', dbname='phytogether', port=5432)
cursor = conn.cursor()


for date in date_list:
    for job in job_list:
        page = 1
        data = ['data']
        while len(data) > 0:
            data = get_stock_log(date, date, job, str(page))
            data_list = [list(d.values()) for d in data]
            save_stock_log(cursor, data_list)
            # time.sleep(3)
            print(date + ' | ' + job + ' | ' + str(page))
            page += 1


conn.commit()
conn.close()

2023-06-12 | arrange | 1
2023-06-12 | in | 1
2023-06-12 | out | 1
2023-06-12 | retin | 1
2023-06-12 | retout | 1
2023-06-12 | trans | 1
2023-06-12 | trans | 2
2023-06-12 | trans | 3
2023-06-12 | trans | 4
2023-06-12 | trans | 5
2023-06-12 | trans | 6
2023-06-12 | trans | 7
2023-06-12 | trans | 8
2023-06-12 | trans | 9
2023-06-12 | trans | 10
2023-06-12 | trans | 11
2023-06-12 | trans | 12
2023-06-12 | trans | 13
2023-06-12 | trans | 14
2023-06-12 | trans | 15
2023-06-12 | trans | 16
2023-06-12 | trans | 17
2023-06-12 | SHOP_REQ | 1


In [4]:
def get_order_info(start_date, end_date):


    partner_key = '73c6ac2e3366905793ab35d5a222d572'
    domain_key = '73c6ac2e33669057718abd505d76aa87366d3eed6643997457a0f9a03797aa9f'
    # start_date = '2023-01-01'
    # end_date = '2023-05-21'
    # job = 'in'
    limit = '100'
    

    url = f'http://api2.cloud.ezadmin.co.kr/ezadmin/function.php?partner_key={partner_key}&domain_key={domain_key}&action=get_order_info&date_type=collect_date&start_date={start_date}&end_date={end_date}'
    res = requests.get(url)

    if res.status_code == 200:
        return res.json()['data']
    else:
        return ValueError

In [5]:
test = get_order_info('2023-06-01', '2023-06-01')

In [7]:
len(test)

10

In [10]:
test[1]

{'pack': '0',
 'seq': '503100',
 'shop_id': '10053',
 'shop_name': '쿠팡',
 'order_id': '11000181774027',
 'order_id_seq': '4935975581',
 'order_id_seq2': '',
 'shop_product_id': '7246571242',
 'product_name': '써큐시안 블러드케어 60캡슐',
 'options': '써큐시안 블러드케어 - 초임계 식물성 알티지 오메가3 콜레스테롤 혈행 홍국 코큐텐 6종 주원료 13가지 복합 기능성, 2+1개',
 'qty': '1',
 'order_name': '서정임',
 'order_mobile': '0502-1767-3028',
 'order_tel': '0502-1767-3028',
 'recv_name': '서정임',
 'recv_mobile': '0502-1767-3028',
 'recv_tel': '0502-1767-3028',
 'recv_address': '경기도 고양시 일산서구 탄현동 1480 탄현마을7단지아파트 704동1510호',
 'recv_zip': '10249',
 'memo': '문 앞',
 'status': '1',
 'order_cs': '0',
 'collect_date': '2023-06-01 09:04:10',
 'order_date': '2023-05-31 14:44:14',
 'trans_date': '',
 'trans_date_pos': '',
 'shopstat_date': '2023-06-01 09:31:17',
 'supply_price': '134268',
 'amount': '151990',
 'extra_money': '0',
 'trans_corp': '',
 'trans_no': '',
 'trans_who': '선불',
 'prepay_price': '0',
 'gift': '',
 'hold': '0',
 'org_seq': '0',
 'deal_no': 